In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
import torch.optim as optim

np.random.seed(8)

In [2]:
class RS(nn.Module):
    
    def __init__(self, n_users, n_hobbies, n_features=10):
        super().__init__()
        n_emb = 5
        self.u_embedding = nn.Embedding(n_users + 1, n_emb)
        self.h_embedding = nn.Embedding(n_hobbies + 1, n_emb)
        self.u_flatten = nn.Flatten()
        self.h_flatten = nn.Flatten()
        self.lin1 = nn.Linear(n_features + 2 * n_emb - 2, 128)
        self.lin2 = nn.Linear(128, 32)
        self.lin3 = nn.Linear(32, 2)
        
        self.batchnorm = nn.BatchNorm1d(128)
        self.relu = nn.ReLU()
        
        self.init_parameters()
        
    def init_parameters(self):
        nn.init.uniform_(self.u_embedding.weight, -1.0, 1.0)
        nn.init.uniform_(self.h_embedding.weight, -1.0, 1.0)
        nn.init.uniform_(self.lin1.weight, -1.0, 1.0)
        nn.init.uniform_(self.lin2.weight, -1.0, 1.0)
        nn.init.uniform_(self.lin3.weight, -1.0, 1.0)
        
    def forward(self, x):
        user = x[:, :1, ...]
        user = user.long()
        user = self.u_embedding(user)
        user = self.u_flatten(user)
        
        hobbie = x[:, 1:2, ...]
        hobbie = hobbie.long()
        hobbie = self.h_embedding(hobbie)
        hobbie = self.h_flatten(hobbie)

        x = torch.cat([user, hobbie, x[:, 2:]], dim=1)
        x = x.float()
        
        x = self.lin1(x)
        x = self.relu(x)
        #x = self.batchnorm(x)
        x = self.lin2(x)
        x = self.lin3(x)
        
        return x

In [3]:
model = RS(20000, 20000)
model(torch.ones(2, 10, dtype=int))

tensor([[32.6996, -6.8458],
        [32.6996, -6.8458]], grad_fn=<AddmmBackward>)

In [4]:
sum([p.numel() for p in model.parameters()])

206892

In [5]:
classificator = pd.read_csv('hobby_data/Classificator_hachaton.csv', delimiter=';')
classificator.drop_duplicates(inplace=True)
print(classificator.shape)
classificator.head()

(623, 3)


,id_классификатора,id_родительского_классификатора,Наименование
0,3002985,NaN,Автошкола
1,3002986,3002985.0,Автоклуб
2,3002987,3002985.0,Правила дорожного движения
3,3002988,3002985.0,Юный водитель
4,3002989,NaN,Архитектура


In [6]:
megaRelation = pd.read_csv('hobby_data/MegaRelation_hackaton.csv', delimiter=';')
megaRelation.drop_duplicates(inplace=True)
print(megaRelation.shape)
megaRelation.head()

(724813, 14)


,id_зачисления,Статус,id_ученика,Дата_создания_записи,id_организации,id_заявления,id_услуги,дата_зачисления,дата_отчисления,причина_перевода,предыдущая_запись_зачисления,следующая_запись_зачисления,Плановая_дата_начала_занятий,Плановая_дата_окончания_занятий
0,1325954,3.0,25969.0,08.06.2015 14:47:19,30,950832.0,39178,08.06.2015 0:00:00,29.02.2016 19:28:12,NaN,NaN,3558114.0,01.09.2014 0:00:00,01.09.2021 0:00:00
1,2879918,3.0,158129.0,29.10.2015 18:35:21,30,989658.0,39178,01.09.2010 0:00:00,29.02.2016 19:23:59,NaN,NaN,3557885.0,01.09.2014 0:00:00,01.09.2021 0:00:00
2,2901258,3.0,304476.0,02.11.2015 14:05:56,30,950764.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558103.0,01.09.2014 0:00:00,01.09.2021 0:00:00
3,2912724,3.0,1139915.0,03.11.2015 14:29:27,30,950786.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558105.0,01.09.2014 0:00:00,01.09.2021 0:00:00
4,2912819,3.0,298767.0,03.11.2015 14:34:15,30,950799.0,39178,09.06.2014 0:00:00,29.02.2016 19:28:11,NaN,NaN,3558106.0,01.09.2014 0:00:00,01.09.2021 0:00:00


In [7]:
services = pd.read_csv('hobby_data/services_hackaton.csv', delimiter=';')
services.drop_duplicates(inplace=True)
print(services.shape)
services.head()

(40972, 9)


,id_услуги,Тип_финансирования,Классификатор_услуги,id_организации,Тип_расписания,Наименование_услуги,Дата_создания,длительность_обучения,единица_длительности
0,39178,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 12:50:32,"7,0000",лет
1,39181,бесплатно,3000001,30,Индивидуальное,Архитектура,17.07.2013 13:04:41,"7,0000",лет
2,212072,бесплатно,3000001,30,Индивидуальное,Архитектура,27.10.2015 15:59:48,"7,0000",лет
3,212517,бесплатно,3000001,30,Индивидуальное,Архитектура,29.10.2015 18:45:49,"7,0000",лет
4,39913,бесплатно,3000003,45,Индивидуальное,Декоративно-прикладное искусство,17.07.2013 20:35:28,"7,0000",лет


In [8]:
org = pd.read_csv('hobby_data/org_hackaton.csv', delimiter=';')
org.drop_duplicates(inplace=True)
print(org.shape)
org.head()

(442, 5)


,id_организации,полное_наименование,краткое_наименование,улица,Метро
0,1,Государственное бюджетное образовательное учре...,ГБОУ СПО г.Москвы «Московская средняя специаль...,NaN,NaN
1,2,Государственное бюджетное профессиональное обр...,"ГБПОУ г. Москвы ""Академия джаза""",улица Трёхгорный Вал,Краснопресненская
2,2,Государственное бюджетное профессиональное обр...,"ГБПОУ г. Москвы ""Академия джаза""",улица Трёхгорный Вал,Улица 1905 года
3,3,Государственное бюджетное образовательном учре...,ГБОУ СПО г.Москвы «Московский театральный колл...,улица Чаплыгина,Красные ворота
4,4,Государственное бюджетное образовательное учре...,ГБОУДОД г.Москвы «МГДМШ им. С.С.Прокофьева»,Токмаков переулок,Курская (Кольцевая)


In [9]:
pupil = pd.read_csv('hobby_data/Pupil_hackaton.csv', delimiter=';')
pupil.drop_duplicates(inplace=True)
print(pupil.shape)
pupil.head()

(2742631, 3)


,id_ученика,возраст,пол
0,1,12.0,Женский
1,2,12.0,Женский
2,3,12.0,Женский
3,4,11.0,Женский
4,5,12.0,Женский


In [10]:
request = pd.read_csv('hobby_data/request_hackaton.csv', delimiter=';')
request.drop_duplicates(inplace=True)
print(request.shape)
request.head()

(468379, 9)


,id_заявления,статус_заявления,дата_создания,оценка_вступительных_испытаний,Испытания_пройдены,Зачисление_разрешено,Зачисление_не_разрешено,Причина_отказа_в_зачислении,id_услуги
0,989578,Ожидание прихода Заявителя на вступительные ис...,02.06.2014 17:25:37,NaN,0.0,0.0,0.0,NaN,39178
1,3571456,Архив,05.11.2015 18:51:59,5.0,1.0,0.0,NaN,NaN,39178
2,3571725,Отказ в зачислении,05.11.2015 19:35:39,NaN,0.0,0.0,1.0,NaN,39178
3,950963,Ожидание прихода Заявителя на вступительные ис...,12.05.2014 16:06:20,NaN,0.0,0.0,0.0,NaN,39178
4,952962,Ожидание прихода Заявителя на вступительные ис...,14.05.2014 18:14:47,NaN,0.0,0.0,0.0,NaN,39178


In [11]:
megaRelation = megaRelation[megaRelation['Статус'] != 2]
megaRelation.shape

(436710, 14)

In [12]:
request = request[request['Зачисление_не_разрешено'] != 1]
request.shape

(467064, 9)

In [13]:
dataset = megaRelation[['id_ученика', 'id_заявления', 'id_услуги', 'id_организации']].merge(
    request[['id_заявления', 'оценка_вступительных_испытаний', ]], 
    on='id_заявления', 
    how='left'
)
dataset.shape

(436710, 5)

In [14]:
dataset = dataset.merge(
    services[['id_услуги', 'Тип_финансирования', 'Тип_расписания', 'Классификатор_услуги']],
    on='id_услуги'
)
dataset.shape

(436710, 8)

In [15]:
dataset = dataset.merge(
    pupil,
    on='id_ученика',
    how='left'
)
dataset.shape

(436710, 10)

In [16]:
dataset = dataset.merge(
    org[['id_организации', 'Метро']].drop_duplicates(subset=['id_организации']),
    on='id_организации',
    how='left'
)
dataset.shape

(436710, 11)

In [17]:
dataset = dataset.merge(
    classificator[['id_классификатора', 'Наименование']],
    left_on='Классификатор_услуги',
    right_on='id_классификатора',
    how='left'
)
dataset.shape

(436710, 13)

In [18]:
dataset['Наименование'].value_counts()

Фортепиано                                                  62213
Живопись                                                    37076
Хоровое пение                                               26327
Хореографическое творчество                                 16092
Гитара                                                      15116
                                                            ...  
Кулинария                                                       1
Музыкальная литература                                          1
Цифровая фотография и компьютерная обработка изображения        1
Диджеинг                                                        1
Пресс-клуб                                                      1
Name: Наименование, Length: 394, dtype: int64

In [19]:
drop_cols = ['оценка_вступительных_испытаний', 'id_заявления', 'id_услуги', 'id_организации', 'Классификатор_услуги']
for col in drop_cols:
    del dataset[col]

In [20]:
dataset.head()

,id_ученика,Тип_финансирования,Тип_расписания,возраст,пол,Метро,id_классификатора,Наименование
0,25969.0,бесплатно,Индивидуальное,14.0,Мужской,Баррикадная,NaN,NaN
1,158129.0,бесплатно,Индивидуальное,18.0,Женский,Баррикадная,NaN,NaN
2,304476.0,бесплатно,Индивидуальное,14.0,Мужской,Баррикадная,NaN,NaN
3,1139915.0,бесплатно,Индивидуальное,14.0,Мужской,Баррикадная,NaN,NaN
4,298767.0,бесплатно,Индивидуальное,13.0,Женский,Баррикадная,NaN,NaN


In [21]:
name_dict = {}
counter = 0
for name in dataset['Наименование'].unique():
    name_dict[name] = counter
    counter += 1

dataset['Наименование_номер'] = dataset['Наименование'].map(name_dict)

In [22]:
for i, col in enumerate(dataset.columns):
    print(i+1, col)

1 id_ученика
2 Тип_финансирования
3 Тип_расписания
4 возраст
5 пол
6 Метро
7 id_классификатора
8 Наименование
9 Наименование_номер


In [23]:
dataset_dict = dataset.to_dict()

In [24]:
interests = [[key, val] for key, val in dataset['Наименование_номер'].value_counts().to_dict().items()]
total = sum([val for key, val in interests])
interests = pd.DataFrame([[key, val / total] for key, val in interests])

begin = max(dataset_dict['Наименование_номер']) + 1

for row in tqdm(dataset.itertuples(), total=len(dataset)):
    current_idx = begin + row[0]
    names = np.random.choice(interests[0], 2, p=interests[1])
    name = names[0] if names[0] != row[8] else names[1]
    dataset_dict['Наименование_номер'].update({current_idx: name})
    dataset_dict['возраст'].update({current_idx: row[4]})
    dataset_dict['пол'].update({current_idx: row[5]})
    dataset_dict['id_ученика'].update({current_idx: row[1]})

    hobby_row = dataset[dataset['Наименование_номер'] == name].sample(1)
    #for row_inner in dataset.sample(frac=1).itertuples():
    #    if row_inner[7] == new_row['Наименование']:
    #        hobby_row = row_inner
    #        break
        
    #for i, col in zip([6, 1, 2], ['Метро', 'Тип_финансирования', 'Тип_расписания']):
    #    new_row[col] = hobby_row[i]
    
    for col in ['Метро', 'Тип_финансирования', 'Тип_расписания', 'id_классификатора']:
        dataset_dict[col].update({current_idx: hobby_row[col].iloc[0]})

    #current_idx = begin + row[0]
    #for col in new_row.:
    #    dataset_dict[col].append(new_row[col])
    #dataset.append(new_row, ignore_index=True)
    
#interests[0]

100%|██████████| 436710/436710 [22:31<00:00, 323.18it/s]


In [25]:
dataset = pd.DataFrame.from_dict(dataset_dict)
dataset.shape

(873420, 9)

In [26]:
dataset['label'] = [1] * int(len(dataset) / 2) + [0] * int(len(dataset) / 2)

In [27]:
dataset.head()

,id_ученика,Тип_финансирования,Тип_расписания,возраст,пол,Метро,id_классификатора,Наименование,Наименование_номер,label
0,25969.0,бесплатно,Индивидуальное,14.0,Мужской,Баррикадная,NaN,NaN,0,1
1,158129.0,бесплатно,Индивидуальное,18.0,Женский,Баррикадная,NaN,NaN,0,1
2,304476.0,бесплатно,Индивидуальное,14.0,Мужской,Баррикадная,NaN,NaN,0,1
3,1139915.0,бесплатно,Индивидуальное,14.0,Мужской,Баррикадная,NaN,NaN,0,1
4,298767.0,бесплатно,Индивидуальное,13.0,Женский,Баррикадная,NaN,NaN,0,1


In [28]:
a_list = dataset.columns.to_list()
index1 = a_list. index("Тип_финансирования")
index2 = a_list. index("id_классификатора")
a_list[index1], a_list[index2] = a_list[index2], a_list[index1]

dataset = dataset[a_list]
dataset.head()

,id_ученика,id_классификатора,Тип_расписания,возраст,пол,Метро,Тип_финансирования,Наименование,Наименование_номер,label
0,25969.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,0,1
1,158129.0,NaN,Индивидуальное,18.0,Женский,Баррикадная,бесплатно,NaN,0,1
2,304476.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,0,1
3,1139915.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,0,1
4,298767.0,NaN,Индивидуальное,13.0,Женский,Баррикадная,бесплатно,NaN,0,1


In [29]:
reverse_name_dict = {val: key for key, val in name_dict.items()}
dataset['Наименование'] = dataset['Наименование_номер'].map(reverse_name_dict)

In [30]:
drop_cols = ['Наименование_номер']
for col in drop_cols:
    del dataset[col]

In [31]:
dataset.head()

,id_ученика,id_классификатора,Тип_расписания,возраст,пол,Метро,Тип_финансирования,Наименование,label
0,25969.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,1
1,158129.0,NaN,Индивидуальное,18.0,Женский,Баррикадная,бесплатно,NaN,1
2,304476.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,1
3,1139915.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,1
4,298767.0,NaN,Индивидуальное,13.0,Женский,Баррикадная,бесплатно,NaN,1


In [21]:
dataset = pd.read_csv('dataset_hobby.csv', index_col=0)
print(dataset.shape)
dataset.head()

(873420, 9)


,id_ученика,id_классификатора,Тип_расписания,возраст,пол,Метро,Тип_финансирования,Наименование,label
0,25969.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,1
1,158129.0,NaN,Индивидуальное,18.0,Женский,Баррикадная,бесплатно,NaN,1
2,304476.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,1
3,1139915.0,NaN,Индивидуальное,14.0,Мужской,Баррикадная,бесплатно,NaN,1
4,298767.0,NaN,Индивидуальное,13.0,Женский,Баррикадная,бесплатно,NaN,1


In [22]:
dataset.isna().sum(axis=0)

id_ученика            154072
id_классификатора     102276
Тип_расписания             0
возраст               154368
пол                   154072
Метро                  70591
Тип_финансирования         0
Наименование          102276
label                      0
dtype: int64

In [23]:
dataset.shape

(873420, 9)

In [24]:
dataset.dropna(subset=['Наименование', 'id_ученика', 'id_классификатора'], inplace=True)
dataset.shape

(627949, 9)

In [25]:
dataset['возраст'] = pd.qcut(dataset['возраст'], q=8).astype(str)

In [26]:
cat_cols = [
    'Тип_финансирования', 
    'Тип_расписания', 
    'пол', 
    'Метро', 
    'Наименование', 
    'возраст'
]

no_cols = [
    'Метро',
    'Наименование'
]
for col in no_cols:
    del dataset[col]
    
cat_cols = list(set(cat_cols) - set(no_cols))

for col in tqdm(cat_cols):
    dummy = pd.get_dummies(dataset[col].astype(str), dummy_na=True)
    dummy.columns = [str(col) + str(c) for c in dummy.columns]
    del dataset[col]
    dataset = pd.concat([
        dataset,
        dummy
    ], axis=1)

100%|██████████| 4/4 [00:00<00:00,  7.57it/s]


In [27]:
dataset.shape

(627949, 22)

In [28]:
user_map = {iD: i for i, iD in enumerate(dataset['id_ученика'].unique())}
item_map = {iD: i for i, iD in enumerate(dataset['id_классификатора'].unique())}

dataset['id_ученика'] = dataset['id_ученика'].map(user_map)
dataset['id_классификатора'] = dataset['id_классификатора'].map(item_map)

In [29]:
class RcDataset(Dataset):
    
    def __init__(self, dataset):
        self.data_cols = dataset.columns.to_list()
        self.data_cols.remove('label')
        self.data_len = len(dataset)
        
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        sample = dataset.iloc[idx]
        input_ = sample[self.data_cols].to_numpy()
        #input_ = torch.FloatTensor(input_)
        label = np.float32(sample['label'])
        #label = torch.LongTensor(sample['label'])
        #if label == 1:
        #    label = torch.LongTensor([0, 1])
        #else:
        #    label = torch.LongTensor([1, 0])

        return input_, label

In [30]:
dataset.reset_index(drop=True, inplace=True)

In [31]:
from sklearn.model_selection import StratifiedKFold

#skf = StratifiedKFold(n_splits=20)
#train_index, test_index = next(skf.split(dataset, dataset['label']))

alpha = 0.05
dataset = dataset.sample(frac=1).reset_index(drop=True)
test_len = int(len(dataset) * alpha)

dataset_train = RcDataset(dataset[:-test_len])
dataloader_train = DataLoader(dataset_train, batch_size=8, shuffle=True)

dataset_test = RcDataset(dataset[-test_len:])
dataloader_test = DataLoader(dataset_test, batch_size=8, shuffle=True)


In [36]:
n_users = dataset['id_ученика'].unique().shape[0]
n_hobbies = dataset['id_классификатора'].unique().shape[0]
n_features = dataset.shape[1]-1 + 2
model = RS(n_users, n_hobbies, n_features)

criterion = nn.BCEWithLogitsLoss()
#criterion = nn.CrossEntropyLoss()
sigmoid = nn.Sigmoid()
softmax = nn.Softmax()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
n_epochs = 10
batch_size = dataloader_train.batch_size

In [48]:
for epoch in range(n_epochs):
    running_loss = 0.
    running_acc = 0.
    for phase in ['train', 'test']:
        if phase == 'train':
            dataloader = dataloader_train
            model.train()
        elif phase == 'test':
            dataloader = dataloader_test
            model.eval()
            
        pbar = tqdm(enumerate(dataloader), total=len(dataloader))
    
        for idx, (inputs, labels) in pbar:
            preds = model(inputs)
            preds = softmax(preds / 100)[:,1]
            loss = criterion(preds, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            acc = (preds.round() == labels).float().mean()
            running_acc += acc
            scale_value = 1 / max(idx, 1)
            running_loss += loss.item()
            pbar.set_description(
                "Epoch: {}/{} Phase: {}, Loss: {:.4f}, Acc: {:.4f}".format(
                    epoch,
                    n_epochs,
                    phase,
                    running_loss * scale_value,
                    running_acc * scale_value
            ))

  0%|          | 0/74569 [00:00<?, ?it/s]<ipython-input-48-8f07ed37d851>:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = softmax(preds / 100)[:,1]
Epoch: 0/10 Phase: train, Loss: 0.6680, Acc: 0.5977:  34%|███▍      | 25169/74569 [57:54<1:53:40,  7.24it/s]


KeyboardInterrupt: 

In [51]:
labels
#preds

tensor([0., 0., 0., 0., 1., 0., 0., 1.])

In [52]:
preds

tensor([4.6306e-02, 2.8886e-01, 2.8606e-01, 2.0857e-05, 1.0003e-03, 1.4651e-04,
        6.9372e-02, 2.2458e-03], grad_fn=<SelectBackward>)

In [68]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
output.backward()


In [68]:
dataset

,id_ученика,id_классификатора,label,Тип_расписанияИндивидуальное,Тип_расписанияОбщее,Тип_расписанияnan,полЖенский,полМужской,полnan,"возраст(0.999, 8.0]",...,"возраст(12.0, 14.0]","возраст(14.0, 15.0]","возраст(15.0, 17.0]","возраст(17.0, 120.0]","возраст(8.0, 10.0]",возрастnan,возрастnan,Тип_финансированиябесплатно,Тип_финансированияплатно,Тип_финансированияnan
0,77128,380,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,126538,368,1,1,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
2,130339,371,1,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,118393,366,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,94383,332,1,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627944,135377,372,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
627945,151384,317,0,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,0,1,0
627946,166568,325,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,0,0
627947,138040,331,0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0


In [69]:
dataset[dataset['id_ученика'] == 118393]

,id_ученика,id_классификатора,label,Тип_расписанияИндивидуальное,Тип_расписанияОбщее,Тип_расписанияnan,полЖенский,полМужской,полnan,"возраст(0.999, 8.0]",...,"возраст(12.0, 14.0]","возраст(14.0, 15.0]","возраст(15.0, 17.0]","возраст(17.0, 120.0]","возраст(8.0, 10.0]",возрастnan,возрастnan,Тип_финансированиябесплатно,Тип_финансированияплатно,Тип_финансированияnan
3,118393,366,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
185934,118393,349,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
210133,118393,305,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
271074,118393,325,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
293376,118393,349,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
360995,118393,349,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [71]:
for idx, row in dataset[dataset['id_ученика'] == 126538].iterrows():
    preds = model(torch.from_numpy(np.expand_dims(row[dataset_test.data_cols].to_numpy(), axis=0)))
    print(row['label'], softmax(preds / 100)[:,1])

1 tensor([0.7101], grad_fn=<SelectBackward>)
0 tensor([0.3647], grad_fn=<SelectBackward>)
0 tensor([1.6582e-07], grad_fn=<SelectBackward>)
1 tensor([0.7101], grad_fn=<SelectBackward>)
1 tensor([0.7101], grad_fn=<SelectBackward>)
0 tensor([0.0118], grad_fn=<SelectBackward>)
1 tensor([0.7101], grad_fn=<SelectBackward>)


<ipython-input-71-d51392264ed2>:3: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print(row['label'], softmax(preds / 100)[:,1])


In [72]:
torch.save(model.state_dict(), 'checkpoint_1.pth')

In [73]:
ls

Analysis.ipynb  checkpoint_1.pth   recognition-service/       Мероприятия.xlsx
books.csv       dataset_hobby.csv  recommendations_hackaton/
books_data/     hobby_data/        Train.ipynb
